<a href="https://colab.research.google.com/github/ghizlanebo/Chest-X-ray-Pathologies-Plassification/blob/main/BertLDa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bertopic[all]

  Using cached https://files.pythonhosted.org/packages/3a/83/e74092e7f24a08d751aa59b37a9fc572b2e4af3918cb66f7766c3affb1b4/transformers-3.5.1-py3-none-any.whl
ERROR: transformers 3.5.1 has requirement sentencepiece==0.1.91, but you'll have sentencepiece 0.1.95 which is incompatible.
ERROR: simpletransformers 0.61.6 has requirement transformers>=4.2.0, but you'll have transformers 3.5.1 which is incompatible.
ERROR: flair 0.7 has requirement sentencepiece<=0.1.91, but you'll have sentencepiece 0.1.95 which is incompatible.
  Found existing installation: transformers 4.6.1
    Uninstalling transformers-4.6.1:
      Successfully uninstalled transformers-4.6.1


In [ ]:
!pip install simpletransformers


  Using cached https://files.pythonhosted.org/packages/d5/43/cfe4ee779bbd6a678ac6a97c5a5cdeb03c35f9eaebbb9720b036680f9a2d/transformers-4.6.1-py3-none-any.whl
ERROR: transformers 4.6.1 has requirement tokenizers<0.11,>=0.10.1, but you'll have tokenizers 0.9.3 which is incompatible.
ERROR: flair 0.7 has requirement sentencepiece<=0.1.91, but you'll have sentencepiece 0.1.95 which is incompatible.
ERROR: flair 0.7 has requirement transformers<=3.5.1,>=3.5.0, but you'll have transformers 4.6.1 which is incompatible.
  Found existing installation: transformers 3.5.1
    Uninstalling transformers-3.5.1:
      Successfully uninstalled transformers-3.5.1


In [ ]:
import os
import pandas as pd
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.graph_objs as go
import gensim
from gensim import corpora, models, similarities
import logging
import tempfile
from nltk.corpus import stopwords
from string import punctuation
from collections import OrderedDict
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

init_notebook_mode(connected=True) #do not miss this line

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df=pd.read_excel('/content/drive/MyDrive/REC_CG_19-21.xlsx')

In [ ]:
corpus=[]
a=[]
for i in range(len(df['descri'])):
        a=df['descri'][i]
        corpus.append(a)
        
corpus[0:10]

['- Concerné(s): MOHAMED & OUMAIMA   - Situation : Non remis en paiement   - Dossier de recherche: Dossier de compagne de controle de maintien de droit   - Clé de recherche: N° Pension   - Délai de retard:   - Réclamations répétitives: Oui  N.B: Scanné/GED Sous Réf N° 1594/18 et affecté le 11/12/2018.',
 'L’intéressé  a réclame date de naissance de enfant  safae erroné.',
 '672,00DH] [ATD DEFENSE',
 'Attestation pension',
 'RAPPEL : [ 9018,52DH] [PREMIER ARRERAGE',
 'aucun',
 'Dde d’info .',
 'RET : [    223,52DH] [ Prec. Prets',
 'DIVORCE',
 'Etat..........: V  Demande validée']

In [ ]:
list1 = ['cmr','bonjour','merci','comme', 'si', 'faire', 'cette', 'aussi', 'ainsi', 'toutes','beaucoup', 'faut', 'falloir',
              'cela', 'va', 'aller', 'là', 'ça','peux', 'pouvoir', 'quand', 'encore', 'donc', 'toujours', 
              'où','quelques', 'autre', 'autres', 'alors', 'entre', 'car', 'après','avant','depuis', 'également', 'un','trois', 'quatre', 'avoir', 'bon', 
              'première', 'premier','deuxième', 'niveau', 'face', 'contre', 'journée', 'début','jours', 
              'journée', 'année', 'jamais', 'lu', 'lire','selon', 'hui', 'aujourd', 'grand','dire', 'dit',
              'dis', 'cet','cette','nouveau','nouvelle','mis','mettre','chez','1','2','3','4','5','6','7','8','9','0',']','[','(',')','#','$',',',':','/','B.C.P','&','dde','ddr','NaN','nan','a','non',"d'",'03/03/2021/',
 '08/04/2021,','08:03:21','08:35:37','10/10/2019,','10:18:57','11994.23','13577.38','13973.','13:05:02','14021.68','16:16:08','1990,35dh][','297905',
 '30862,14dh]',
 '592110076',
 '7140086/111',
 '7182690',
 '7293108',
 '7303271/312',
 '7303271/313',
 '9397.47',
 '[nais.rajaa',
 ']&#9474;[20/03/2000]',
 'aafecte',
 'augment',
 'b.c.p.',
 'cpc',
 'elhamdounifatima8@gmail.com',
 'jed!',"l'",'aucun','ctrl','réc','rec','rev','cs','n°','réf','01/02/2020','cdt','prière','bonjour','cordialement','&#9474;date','11/02/2020','11/12/2019','2019','2020','2021','2016','2015','2017','2016','2018','rcar','el','1594/18','672,00dh]', '[atd','11/12/2018.','etat..........:', 'v','ret', '223,52dh]', 'prec.','safae','oumaima','mohamed','oui',
  'n.b:','sous','9018,52dh']

In [ ]:
import nltk
nltk.download('stopwords')
stoplist = stopwords.words('french') + list(punctuation) + list1

texts = [[word for word in str(document).lower().split() if word not in stoplist] for document in corpus]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
texts[:3]

[['concerné(s):',
  'situation',
  'remis',
  'paiement',
  'dossier',
  'recherche:',
  'dossier',
  'compagne',
  'controle',
  'maintien',
  'droit',
  'clé',
  'recherche:',
  'pension',
  'délai',
  'retard:',
  'réclamations',
  'répétitives:',
  'scanné/ged',
  'affecté'],
 ['l’intéressé', 'réclame', 'date', 'naissance', 'enfant', 'erroné.'],
 ['defense']]

In [ ]:
pip install transformers -U

Requirement already up-to-date: transformers in /usr/local/lib/python3.7/dist-packages (4.6.1)
     |████████████████████████████████| 3.3MB 3.8MB/s 
ERROR: flair 0.7 has requirement sentencepiece<=0.1.91, but you'll have sentencepiece 0.1.95 which is incompatible.
ERROR: flair 0.7 has requirement transformers<=3.5.1,>=3.5.0, but you'll have transformers 4.6.1 which is incompatible.
  Found existing installation: tokenizers 0.9.3
    Uninstalling tokenizers-0.9.3:
      Successfully uninstalled tokenizers-0.9.3


In [ ]:
from simpletransformers.language_representation import RepresentationModel


In [ ]:
model = RepresentationModel(
        model_type="bert",
        model_name="bert-base-multilingual-cased",
        use_cuda=False
    )

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTextRepresentation: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTextRepresentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTextRepresentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [ ]:
sentence_vectors = model.encode_sentences(texts, combine_strategy="mean")

In [ ]:
from bertopic import BERTopic


In [ ]:
s=str(texts)

In [ ]:
a=df['descri']

In [ ]:
topic_model = Topic(embedding_model="camembert-base").fit(a)

Exception when trying to download http://sbert.net/models/camembert-base.zip. Response 404
SentenceTransformer-Model http://sbert.net/models/camembert-base.zip not found. Try to create it from scratch
Try to create Transformer Model camembert-base with mean pooling


Downloading:   0%|          | 0.00/508 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/811k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.40M [00:00<?, ?B/s]